In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_black
%load_ext nb_mypy

In [ ]:
import nbimporter
from Exercise04AI import Exercise04AI
from Exercise06AI import Exercise06AI
from Exercise07AI import Exercise07AI

# Aufgabe 08: Minimax mit Alpha-Beta-Pruning, Memoisierung, Progressive Deepening und Singular Value Extension

Dieses Notebook erweitert den Minimax-Algorithmus mit Alpha-Beta-Pruning und Memoisierung um das Progressive Deepening.

In [ ]:
import chess
from typing import Any


class Exercise08AI(Exercise07AI):
    """Chooses middle game moves using minimax algorithm, alpha-beta-pruning,
    memoization and progressive deepening and the quiescence search"""

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        self.real_depth: int = 0

    def reset(self) -> None:
        """Resets all internal variables"""
        super().reset()
        self.real_depth = 0

In [ ]:
from typing import Any, Callable


class Exercise08AI(Exercise08AI):  # type: ignore
    @staticmethod
    def memoize_minimax(minimax: Callable):
        def minimax_memoized(
            self,
            board: chess.Board,
            depth: int,
            current_evaluation: int,
            alpha: int = -Exercise08AI.LIMIT,
            beta: int = Exercise08AI.LIMIT,
            real_depth: int = 0,
        ):
            key = Exercise04AI.get_key(board, depth)
            self.stats[-1]["cache_tries"] += 1
            if key in self.cache:
                self.stats[-1]["cache_hits"] += 1
                return self.get_from_cache(
                    minimax,
                    key,
                    board,
                    depth,
                    current_evaluation,
                    alpha,
                    beta,
                    real_depth,
                )
            result = minimax(
                self, board, depth, current_evaluation, alpha, beta, real_depth
            )
            self.store_in_cache(key, result, alpha, beta)
            return result

        return minimax_memoized

    def get_from_cache(
        self,
        minimax: Callable,
        key: tuple,
        board: chess.Board,
        depth: int,
        current_eval: int,
        alpha: int,
        beta: int,
        real_depth: int,
    ) -> tuple:
        """Gets a result from the cache if possible."""
        flag, evaluation, move = self.cache[key]
        if flag == "=":
            return evaluation, move
        elif flag == "≤":
            if evaluation <= alpha:
                return evaluation, move
            elif evaluation < beta:
                result = minimax(
                    self, board, depth, current_eval, alpha, evaluation, real_depth
                )
                self.store_in_cache(key, result, alpha, evaluation)
                return result
            else:
                result = minimax(
                    self, board, depth, current_eval, alpha, beta, real_depth
                )
                self.store_in_cache(key, result, alpha, beta)
                return result
        else:
            if evaluation <= alpha:
                result = minimax(
                    self, board, depth, current_eval, alpha, beta, real_depth
                )
                self.store_in_cache(key, result, alpha, beta)
                return result
            elif evaluation < beta:
                result = minimax(
                    self, board, depth, current_eval, evaluation, beta, real_depth
                )
                self.store_in_cache(key, result, evaluation, beta)
                return result
            else:
                return evaluation, move

In [ ]:
from typing import Any
import heapq


class Exercise08AI(Exercise08AI):  # type: ignore
    @Exercise08AI.memoize_minimax
    def minimax(
        self,
        board: chess.Board,
        depth: int,
        current_evaluation: int,
        alpha: int = -Exercise07AI.LIMIT,
        beta: int = Exercise07AI.LIMIT,
        real_depth: int = 0,
    ) -> tuple[int, chess.Move | None]:
        """Searches the best value with given depth using minimax algorithm"""
        real_depth += 1
        early_abort_evaluation = self.minimax_early_abort(
            board, depth, current_evaluation
        )
        if early_abort_evaluation is not None:
            self.real_depth = real_depth
            return early_abort_evaluation, None

        best_move = None

        # White to play (positive numbers are good)
        moves: list[tuple[int, int, chess.Move]] = []
        if board.turn:
            for i, move in enumerate(board.legal_moves):
                key = Exercise04AI.get_key(board, depth - 2)
                old_eval = self.cache.get(key, (None, i))[1]
                heapq.heappush(moves, (-old_eval, i, move))
            maxEvaluation = alpha
            while moves:
                move = heapq.heappop(moves)[2]
                board.push(move)
                evaluation, _ = self.minimax(
                    board,
                    depth - 1,
                    self.evaluate(board, current_evaluation),
                    maxEvaluation,
                    beta,
                    real_depth,
                )
                board.pop()
                if evaluation >= beta:
                    return evaluation, move
                if depth == self.DEPTH and evaluation > maxEvaluation:
                    best_move = move
                maxEvaluation = max(maxEvaluation, evaluation)
            return maxEvaluation, best_move

        # Black to play (negative numbers are good)
        else:
            for i, move in enumerate(board.legal_moves):
                key = Exercise04AI.get_key(board, depth - 2)
                old_eval = self.cache.get(key, (None, i))[1]
                heapq.heappush(moves, (old_eval, i, move))
            minEvaluation = beta
            while moves:
                move = heapq.heappop(moves)[2]
                board.push(move)
                evaluation, _ = self.minimax(
                    board,
                    depth - 1,
                    self.evaluate(board, current_evaluation),
                    alpha,
                    minEvaluation,
                    real_depth,
                )
                board.pop()
                if evaluation <= alpha:
                    return evaluation, move
                if depth == self.DEPTH and evaluation < minEvaluation:
                    best_move = move
                minEvaluation = min(minEvaluation, evaluation)
            return minEvaluation, best_move

In [ ]:
class Exercise08AI(Exercise08AI):  # type: ignore
    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """Gets the best next move"""
        best_move = super().get_next_middle_game_move(board)
        self.stats[-1]["eval_depth"] = self.real_depth
        return best_move

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
import Exercise03AI as Exercise03AITests
import Exercise04AI as Exercise04AITests

In [ ]:
# Create player and board
unit_test_player = Exercise08AI(player_name="Ex08AI", search_depth=2)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
board

In [ ]:
# Test minimax
Exercise04AITests.test_minimax(unit_test_player, board)

In [ ]:
# Test next move function (with memoized minimax result)
Exercise03AITests.test_next_move(unit_test_player, board)

## Temporärer Bereich

Der folgende Bereich dient zum temporären Debuggen und kann nicht-funktionierenden Code enthalten. Dieser Bereich wird vor der Abgabe entfernt.